In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import pickle
from keras_facenet import FaceNet
from datetime import datetime

# Load trained model and label encoder
embedder = FaceNet()
knn = pickle.load(open("knn_model.pkl", "rb"))
label_encoder = pickle.load(open("label_encoder.pkl", "rb"))

# Dictionary to track attendance and face persistence
attendance_log = {}
face_persistence = {}  # To store persistent face info: {face_id: {"person_id": ..., "last_seen": ..., "confidence": ...}}
confidence_threshold = 0.65  # Threshold for recognizing a face
persistence_timeout = 2  # Seconds to persist a face ID if not detected

unknown_faces_dir = "Unknown_Faces"
os.makedirs(unknown_faces_dir, exist_ok=True)

# Load student details from CSV
df = pd.read_csv("Attendance.csv")
student_info = {row["Person ID"]: row for _, row in df.iterrows()}

# Track last snapshot time for unknown faces
unknown_face_last_seen = {}
snapshot_interval = 5  # Minimum seconds between snapshots

# Start webcam
cap = cv2.VideoCapture(0)

def calculate_iou(box1, box2):
    """Calculate Intersection over Union (IoU) for two bounding boxes."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = embedder.extract(rgb_frame, threshold=0.95)

    current_time = datetime.now()
    timestamp_text = current_time.strftime("%Y-%m-%d %H:%M:%S")

    # Track current frame's faces
    current_faces = {}

    for face_idx, face in enumerate(faces):
        embedding = face["embedding"].reshape(1, -1)
        distances, indices = knn.kneighbors(embedding, n_neighbors=3)  # Get top 3 predictions

        # Find the closest matching face from the previous frame using IoU
        best_match_id = None
        best_iou = 0
        current_box = face["box"]

        for prev_face_id, prev_info in list(face_persistence.items()):
            prev_box = prev_info["box"]
            iou = calculate_iou(current_box, prev_box)
            if iou > best_iou and iou > 0.5:  # IoU threshold for matching
                best_iou = iou
                best_match_id = prev_face_id

        # If a match is found, use the previous face ID; otherwise, assign a new one
        if best_match_id is not None:
            face_id = best_match_id
        else:
            face_id = f"face_{len(face_persistence) + 1}"

        # Check if the closest match is within the confidence threshold
        if distances[0][0] < confidence_threshold:
            pred_id = knn.predict(embedding)[0]
            person_id = label_encoder.inverse_transform([pred_id])[0]
            confidence = distances[0][0]

            # Update or initialize face persistence
            if face_id in face_persistence:
                # Update only if the new confidence is better or the person_id is the same
                prev_info = face_persistence[face_id]
                if prev_info["person_id"] == person_id or confidence < prev_info["confidence"]:
                    face_persistence[face_id] = {
                        "person_id": person_id,
                        "confidence": confidence,
                        "last_seen": current_time,
                        "box": current_box
                    }
            else:
                face_persistence[face_id] = {
                    "person_id": person_id,
                    "confidence": confidence,
                    "last_seen": current_time,
                    "box": current_box
                }

            # Fetch student details
            student = student_info.get(person_id, {})
            name = student.get("Name", "Unknown")
            dept = student.get("Department", "Unknown")
            roll_no = student.get("Roll No", "Unknown")

            now_str = current_time.strftime("%Y-%m-%d %H:%M:%S")
            
            # Log attendance for known person
            if person_id not in attendance_log:
                attendance_log[person_id] = {
                    "Person ID": person_id,
                    "Name": name,
                    "Roll No": roll_no,
                    "Department": dept,
                    "Entry": now_str,
                    "Exit": None
                }
            else:
                attendance_log[person_id]["Exit"] = now_str

            x, y, w, h = face["box"]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{name}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        else:
            # Handle unknown faces (do not log in attendance)
            now_str = current_time.strftime("%Y-%m-%d_%H-%M-%S")
            face_key = f"unknown_{face_idx}"
            
            if face_key not in unknown_face_last_seen or (current_time - unknown_face_last_seen[face_key]).seconds > snapshot_interval:
                snapshot = frame.copy()
                cv2.putText(snapshot, now_str, (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                img_path = os.path.join(unknown_faces_dir, f"unknown_{now_str}.jpg")
                cv2.imwrite(img_path, snapshot)
                unknown_face_last_seen[face_key] = current_time
            
            x, y, w, h = face["box"]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, "Unknown", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Update current faces
        current_faces[face_id] = face_persistence.get(face_id, {"box": current_box})

    # Update face persistence: remove faces not seen for too long
    face_persistence = {
        face_id: info for face_id, info in face_persistence.items()
        if (current_time - info["last_seen"]).total_seconds() < persistence_timeout
        or face_id in current_faces
    }

    # Add timestamp on bottom-left corner
    cv2.putText(frame, timestamp_text, (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv2.imshow("Face Recognition Attendance", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Save attendance log
attendance_df = pd.DataFrame.from_dict(attendance_log, orient="index")
attendance_df = attendance_df[["Person ID", "Name", "Roll No", "Department", "Entry", "Exit"]]
attendance_df.to_csv("Attendance_Report.csv", index=False)
print("Attendance log saved.")

c:\Users\shree\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\shree\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


KeyError: "None of [Index(['Person ID', 'Name', 'Roll No', 'Department', 'Entry', 'Exit'], dtype='object')] are in the [columns]"